In [1]:
import pandas as pd

In [3]:
movies = pd.read_csv('datasets/movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
ratings = pd.read_csv('datasets/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,17,4.0,944249077
1,1,25,1.0,944250228
2,1,29,2.0,943230976
3,1,30,5.0,944249077
4,1,32,5.0,943228858


In [5]:
tags = pd.read_csv('datasets/tags.csv')
tags.head()

,userId,movieId,tag,timestamp
0,22,26479,Kevin Kline,1583038886
1,22,79592,misogyny,1581476297
2,22,247150,acrophobia,1622483469
3,34,2174,music,1249808064
4,34,2174,weird,1249808102


In [6]:
links = pd.read_csv('datasets/links.csv')
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [8]:
from neo4j import GraphDatabase
import pandas as pd

URI = "bolt://localhost:7687"
USER = "neo4j"
PASSWORD = "Saleh.4992"

class MovieGraph:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def add_movies(self, movies_df):
        with self.driver.session() as session:
            for _, row in movies_df.iterrows():

                genres = row['genres'].split('|')
                
                session.execute_write(self._create_movie_and_genres, row['title'], row['movieId'], genres)

    @staticmethod
    def _create_movie_and_genres(tx, title, movie_id, genres):
        tx.run("MERGE (m:Movie {id: $id, title: $title})", id=movie_id, title=title)
        
        for genre_name in genres:
            tx.run("""
                MERGE (g:Genre {name: $name})
                WITH g
                MATCH (m:Movie {id: $id})
                MERGE (m)-[:IN_GENRE]->(g)
            """, name=genre_name, id=movie_id)


df = pd.read_csv('datasets/movies.csv')

app = MovieGraph(URI, USER, PASSWORD)
print("white")
app.add_movies(df.head(100))
app.close()
print("end")

white
end
